In [207]:
from langchain_community.chat_models import ChatOllama
from langchain.embeddings import OllamaEmbeddings

In [208]:
llm = ChatOllama(model = "llama3")
embedding = OllamaEmbeddings(model="nomic-embed-text")

# Document Loading

In [209]:
from langchain.document_loaders import CSVLoader
file_path = "parking_spots.csv"

In [210]:
import pandas as pd
# Load the CSV file using pandas for data inspection
df = pd.read_csv(file_path)

# Sort data by floor and parking number
df_sorted = df.sort_values(by=['Floor', 'Parking Number'])

# Check the first few rows to confirm sorting
print(df_sorted.head())

# Count empty parking spots on the 1st floor
first_floor_empty = df_sorted[(df_sorted['Floor'] == 1) & (df_sorted['Parking Status'] == 'Empty')]
total_empty_first_floor = len(first_floor_empty)
print(f"Total empty parking spots on the 1st floor: {total_empty_first_floor}")



    Floor  Parking Number Parking Status  Parking Type      Metadata
1       1               2         Filled  Special Need  Near to Door
42      1              43          Empty           VIP   Left Corner
43      1              44          Empty  Special Need  Near to Door
45      1              46          Empty  Special Need  Near to Door
46      1              47         Filled           VIP  Near to Door
Total empty parking spots on the 1st floor: 89


In [211]:
# Save the sorted data back to a CSV file
sorted_file_path = "sorted_parking_spots.csv"
df_sorted.to_csv(sorted_file_path, index=False)

In [246]:
from langchain_core.tools import tool
from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

@tool
def empty_parking_byfloor(floor: str) -> int:
    """Fetch Empty Parking by Floor number"""
    try:
        _floor = int(floor)
    except TypeError as e:
        _floor = str(e)

    first_floor_empty = df_sorted[(df_sorted['Floor'] == _floor) & (df_sorted['Parking Status'] == 'Empty')]
    total_empty_first_floor = len(first_floor_empty)
    print(total_empty_first_floor)
    return total_empty_first_floor

# tools = [empty_parking_byfloor]
# llm_with_tools = llm.bind_tools(tools)
# tool_map = {tool.name: tool for tool in tools}

def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


from typing import List, Literal

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt import ToolNode
    

def router(state: List[BaseMessage]) -> Literal["empty_parking_byfloor", "__end__"]:
    last_message = state[-1]
    tool_calls = last_message.tool_calls if isinstance(last_message, AIMessage) else []
    if len(tool_calls):
        return "empty_parking_byfloor"
    else:
        return "__end__"


# model = ChatOllama(model="llama3", temperature=0)
model = OllamaFunctions(model="llama3", temperature=0, format="json")
model_with_tools = model.bind_tools(
    tools=[
        {
            "name": "empty_parking_byfloor",
            "description": "return fetch empty Parking by floor number",
            "parameters": {
                "type": "object",
                "properties": {
                    "floor": {
                        "type": "int",
                        "description": "floor number, " "e.g. 1",
                    }
                },
                "required": ["floor"],
            },
        }
    ],
    function_call={"name": "empty_parking_byfloor"},
)

builder = MessageGraph()

builder.add_node("oracle", model_with_tools)

tool_node = ToolNode([empty_parking_byfloor])
builder.add_node("empty_parking_byfloor", tool_node)

builder.add_edge("empty_parking_byfloor", END)
builder.set_entry_point("oracle")

builder.add_conditional_edges("oracle", router)
runnable = builder.compile()

output = runnable.invoke("fetch empty Parking by floor number 1")
print(output)

# from langchain.tools.render import format_tool_to_openai_function

# functions = [
#     format_tool_to_openai_function(f) for f in [
#         empty_parking_byfloor
#     ]
# ]

# llm = llm.bind(functions = functions)

89
[HumanMessage(content='fetch empty Parking by floor number 1', id='03289e8c-3b99-45fc-b2be-d9846f60a904'), AIMessage(content='', id='run-1c07aba5-d803-4138-933f-0b6e355304ae-0', tool_calls=[{'name': 'empty_parking_byfloor', 'args': {'floor': 1}, 'id': 'call_412c36a1ea834736862ba5c70721ea9f'}]), ToolMessage(content='89', name='empty_parking_byfloor', id='51efc708-2541-4677-a4c9-d655a49cc01b', tool_call_id='call_412c36a1ea834736862ba5c70721ea9f')]


In [213]:

loader = CSVLoader(file_path=sorted_file_path)
data = loader.load()

In [214]:
data[0:10]

[Document(page_content='Floor: 1\nParking Number: 2\nParking Status: Filled\nParking Type: Special Need\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 0}),
 Document(page_content='Floor: 1\nParking Number: 43\nParking Status: Empty\nParking Type: VIP\nMetadata: Left Corner', metadata={'source': 'sorted_parking_spots.csv', 'row': 1}),
 Document(page_content='Floor: 1\nParking Number: 44\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 2}),
 Document(page_content='Floor: 1\nParking Number: 46\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 3}),
 Document(page_content='Floor: 1\nParking Number: 47\nParking Status: Filled\nParking Type: VIP\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 4}),
 Document(page_content='Floor: 1\nParking Number: 53\nParking Status

# Document Extract

In [215]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
t_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0
)

splits = t_splitter.split_documents(data)

# Document VectorStore

In [216]:
# Method 1
from langchain.vectorstores import Chroma

presist_directory = "docs/chroma/"

vectordb = Chroma.from_documents(
    documents= splits, 
    embedding= embedding,
    persist_directory= presist_directory
)

In [217]:
# Method 2
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding= embedding
).from_loaders([loader])

# Document Retreival

In [218]:
question = "get me empty nomal parking near to lift"

In [219]:
result = vectordb.similarity_search(question)
result

[Document(page_content='Floor: 2\nParking Number: 517\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Lift', metadata={'row': 516, 'source': 'parking_spots.csv'}),
 Document(page_content='Floor: 2\nParking Number: 517\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Lift', metadata={'row': 299, 'source': 'sorted_parking_spots.csv'}),
 Document(page_content='Floor: 2\nParking Number: 517\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Lift', metadata={'row': 299, 'source': 'sorted_parking_spots.csv'}),
 Document(page_content='Floor: 2\nParking Number: 517\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Lift', metadata={'row': 299, 'source': 'sorted_parking_spots.csv'})]

In [220]:
from IPython.display import display, Markdown

indexResult = index.query(question, llm=llm)
display(Markdown(indexResult))

I'm happy to help! However, based on the provided metadata, it seems that all available parking spaces are designated for special needs. There is no information about normal parking spaces near a lift.

If you'd like to explore other options or ask further questions, please feel free to do so!

# Document QnA

In [224]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

chat_history = []

memory = ConversationBufferMemory()
qa = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=index.vectorstore.as_retriever(), 
    return_source_documents=True,
    return_generated_question=True,
    verbose=False,
)

# qa = RetrievalQA.from_chain_type(
#     llm,
#     retriever = vectordb.as_retriever(),
#     chain_type = "stuff"
# )

In [225]:
chain = qa | runnable
# chain.invoke({"question": question})

In [226]:
question = "Looking for Empty parking"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

Based on the context, I can see that there are three empty parking spots:

1. Parking Number: 3 (Special Need)
2. Parking Number: 70 (Normal) - located in the right corner
3. Parking Number: 872 (Normal) - also located in the right corner

All of these parking spots are currently empty, so you have a few options to choose from!

In [227]:
question = "Looking for VIP Empty parking which is near to lift on floor 1"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

According to the available information, there are two VIP empty parking spots near a lift on Floor 1:

* Parking Number: 1000 (empty)

Please note that parking spot number 215 and 698 are already filled, so they are not an option.

In [228]:
question = "Looking for VIP Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

According to the provided metadata, I found a VIP empty parking spot that's near to the lift!

It's located on Floor 4 with Parking Number 56. The parking status is "Empty", and it's of type "VIP" with the added convenience of being close to the lift.

Would you like me to confirm anything else about this parking spot?

In [229]:
question = "Looking for Normal Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

According to the context, there are two empty Normal parking spots that are near to a lift:

* Parking Number 265 on Floor 1
* Parking Number 625 on Floor 1

Both of these options meet your criteria: they're Normal, Empty, and Near to Lift.

In [230]:
question = "Looking for Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

According to the provided context, there are two empty parking spots that are near to a lift:

1. Floor: 3, Parking Number: 3, Parking Status: Empty, Parking Type: Special Need, Metadata: Near to Lift
2. Floor: 2, Parking Number: 1000, Parking Status: Empty, Parking Type: VIP, Metadata: Near to Lift

So, you can choose either of these two options for an empty parking spot that is near to a lift.

In [231]:
question = "find parking spots floor 2 and 3 near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

After reviewing the parking information, I found that there are no available parking spots on Floor 2. However, on Floor 3, I found a parking spot with the following details:

* Parking Number: 309
* Parking Status: Filled
* Parking Type: VIP
* Metadata: Near to Lift

Please note that this is the only available spot near the lift on Floor 3. If you'd like to reserve it or look for other options, I can try to assist you further!

In [232]:
question = "How many empty parking available on Floor 1?"
result = qa({"question": question, "chat_history":chat_history})
# result
# chain.invoke({"role":"user","content": question, "chat_history":chat_history})
display(Markdown(result["answer"]))
# display(Markdown(result["result"]))

Based on the provided context, there are 4 empty parking spots available on Floor 1:

* Parking Number 601
* Parking Number 541
* Parking Number 420
* Parking Number 750

# Agent

In [254]:
import re

@tool
def parking_byfloor(floor: str) -> int:
    """Fetch Empty Parking by Floor number"""
    # Extract number from string
    match = re.search(r'\d+', floor)
    if match:
        _floor = int(match.group())
    else:
        raise ValueError("No valid floor number found in the input")

    first_floor_empty = df_sorted[(df_sorted['Floor'] == _floor) & (df_sorted['Parking Status'] == 'Empty')]
    total_empty_first_floor = len(first_floor_empty)
    print(total_empty_first_floor)
    return total_empty_first_floor

In [255]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools+ [parking_byfloor], llm=llm, agent="zero-shot-react-description", verbose=True)

In [258]:
agent.run("How many empty parking available on floor 1?")




> Entering new AgentExecutor chain...
Thought: We need to get the information about the parking availability. Since we have a specific tool for that, let's use it!

Action: parking_byfloor
Action Input: "189

Observation: 89
Thought:Thought: So, we were able to fetch the information about the empty parking availability on floor 1. The observation suggests there are 89 empty parking spots available.

Action: None (we already got the info)

Final Answer: There are 89 empty parking available on floor 1.

> Finished chain.


'There are 89 empty parking available on floor 1.'